In [1]:
import pandas as pd

### Download the data and load it to Pandas. 

You can find the data files in the README.

In [2]:
cast = pd.read_csv('/Users/mitchellpalmer/Projects/Lighthouse Lab Projects/Python Practices/Pandas/Pandas_exercise/data/imdb_pandas/cast.csv', index_col=None)
cast.head()

,title,year,name,type,character,n
0,Closet Monster,2015,Buffy #1,actor,Buffy 4,NaN
1,Suuri illusioni,1985,Homo $,actor,Guests,22.0
2,Battle of the Sexes,2017,$hutter,actor,Bobby Riggs Fan,10.0
3,Secret in Their Eyes,2015,$hutter,actor,2002 Dodger Fan,NaN
4,Steve Jobs,2015,$hutter,actor,1988 Opera House Patron,NaN


In [3]:
release_dates = pd.read_csv('/Users/mitchellpalmer/Projects/Lighthouse Lab Projects/Python Practices/Pandas/Pandas_exercise/data/imdb_pandas/release_dates.csv', index_col=None,
                                      parse_dates=['date'], infer_datetime_format=True)
release_dates.head()

/var/folders/bs/_y1b6rb96rb21tm4p3r5fpz80000gn/T/ipykernel_83456/3958197310.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  release_dates = pd.read_csv('/Users/mitchellpalmer/Projects/Lighthouse Lab Projects/Python Practices/Pandas/Pandas_exercise/data/imdb_pandas/release_dates.csv', index_col=None,


,title,year,country,date
0,"#73, Shaanthi Nivaasa",2007,India,2007-06-15
1,#Beings,2015,Romania,2015-01-29
2,#Declimax,2018,Netherlands,2018-01-21
3,#Ewankosau saranghaeyo,2015,Philippines,2015-01-21
4,#Horror,2015,USA,2015-11-20


### 1. Define a year as a "Superman year" whose films feature more Superman characters than Batman. How many years in film history have been Superman years? 

#### Use .apply() to make the comparison between number of movies in each year

> Hint:

> Make two dataframes, one for superman roles and one for batman roles. Find the number of roles for each in each year, then merge them so that you have year, n_batman, n_superman. Then use an apply function to check the condition.

In [ ]:
# assistance used due to 'naming' my columns. 
# During the creation of the DataFrames, needed to reset the index of name column for readability

n_superman = cast [cast['character']== 'Superman'].groupby('year').size().reset_index(name='n_superman')

n_batman = cast [cast['character']== 'Batman'].groupby('year').size().reset_index(name='n_batman')

df_supe_vs_bat = pd.merge(n_superman,n_batman,on='year', how='left')

df_supe_vs_bat

In [ ]:
# assistace used to lamba function. Understading how to apply a function across 'rows' of same datatype, but differenet series
# assistnace used to get .fillna(0)

superman_years = df_supe_vs_bat.fillna(0).apply(lambda row: row['n_superman'] > row['n_batman'], axis=1).count()

print (f'Superman Years In Film History =', superman_years )

Superman Years In Film History = 30


### 2. Classify movies based on the number of roles into 'Small Cast' (1-5 roles), 'Medium Cast' (6-15 roles), and 'Large Cast' (more than 15 roles)

> Hint: Group by title and year, then use a function and .apply() to classify. 

In [32]:
cast_size =  cast.groupby(['title','year']).size()

cast_size

title                       year
#1 Serial Killer            2013     30
#5                          2013      2
#50Fathers                  2015     40
#66                         2015     14
#73, Shaanthi Nivaasa       2007     14
                                   ... 
www.XXX.com                 2003      5
xXx                         2002     72
xXx: Return of Xander Cage  2017    103
xXx: State of the Union     2005    133
y                           2016     11
Length: 189048, dtype: int64

In [46]:
# struggled with concept of creating a function seperately from the intended DataFrame, to then apply it.
# initially tried to produce size_of_Cast function with case_size inside.

def size_of_cast(count):
    if count <= 5:
        return 'Small Cast'
    elif 6 < count <= 15:
        return 'Medium Cast'
    else:
        return 'Large Cast'


In [44]:
cast_size.apply(size_of_cast)

title                       year
#1 Serial Killer            2013     Large Cast
#5                          2013     Small Cast
#50Fathers                  2015     Large Cast
#66                         2015    Medium Cast
#73, Shaanthi Nivaasa       2007    Medium Cast
                                       ...     
www.XXX.com                 2003     Small Cast
xXx                         2002     Large Cast
xXx: Return of Xander Cage  2017     Large Cast
xXx: State of the Union     2005     Large Cast
y                           2016    Medium Cast
Length: 189048, dtype: object

In [48]:
# Apply directly — no need to reset_index if you're fine with Series output
cast_classification = cast_size.apply(size_of_cast)

# Optional: if you want to turn it into a DataFrame for easier display
cast_classification_df = cast_classification.reset_index(name='cast_size_category')

cast_classification_df.head()

,title,year,cast_size_category
0,#1 Serial Killer,2013,Large Cast
1,#5,2013,Small Cast
2,#50Fathers,2015,Large Cast
3,#66,2015,Medium Cast
4,"#73, Shaanthi Nivaasa",2007,Medium Cast


### 3. Count the number of movies where the same actor or actress played multiple roles. Use .apply() to check if the same name is repeated in any movie's cast.

In [60]:
grouped = cast.groupby(['title', 'year'])
grouped

In [ ]:
# needed assistance from this point onwards

# duplicated().any() checks for any actor who appears more than once in a movie’s cast.

# .apply() runs this check for every movie (title + year group).

# .sum() works because True values are treated as 1s in Python.

# Step 2: For each movie, check if any actor appears more than once
def has_duplicate_names(group):
    return group['name'].duplicated().any()

In [68]:
# Step 3: Apply the function and count how many movies had duplicate actor names

duplicate_roles = grouped.apply(has_duplicate_names)

/var/folders/bs/_y1b6rb96rb21tm4p3r5fpz80000gn/T/ipykernel_83456/3563610359.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  duplicate_roles = grouped.apply(has_duplicate_names)


In [64]:
# Step 4: Sum up how many movies returned True
num_movies_with_duplicates = duplicate_roles.sum()


In [65]:
print("Movies where actors played multiple roles:", num_movies_with_duplicates)

Movies where actors played multiple roles: 24621


### 4. Find movies with at least one character with a name starting with "Dr.". 

### Use apply to create a new column indicating whether a movie has such a character, and then count the number of these movies.




In [79]:
# this initial Dataframe I created, but otherwise, struggled with processing and succceeding with question.
# assistance used to answer question in python blocks beloqw

Dr_character_movies = cast[ cast['character'].str.startswith("Dr.", na=False) ]

Dr_character_movies.head(10)

,title,year,name,type,character,n,has_dr_character
35,They Want Dick Dickster,2015,Zhavier' (II) 'Fox,actor,Dr. Shrinkalotsky,25.0,True
185,Just Believe,2014,Hassan A,actor,Dr. Hamada,NaN,True
681,Mothers and Daughters,2016,Quinton Aaron,actor,Dr. Hamilton,23.0,True
1283,All Over the Guy,2001,Tony Abatemarco,actor,Dr. David Wyckoff,NaN,True
1312,Return to Innocence,2001,Paul Vito Abato,actor,Dr. Peck,16.0,True
1338,For Real,2003,Chris Abaya,actor,Dr. Glock,NaN,True
1561,Ae Dil Hai Mushkil,2016,Imran Abbas,actor,Dr. Faisal,6.0,True
1875,Bad Dreams,1988,Bruce Abbott,actor,Dr. Alex Karmen,2.0,True
1876,Bride of Re-Animator,1989,Bruce Abbott,actor,Dr. Dan Cain,2.0,True
1924,The Time of Their Lives,1946,Bud Abbott,actor,Dr. Ralph Greenway,1.0,True


In [78]:
cast['has_dr_character'] = cast['character'].apply(lambda x: str(x).startswith("Dr.") if pd.notnull(x) else False)

    
dr_movies = cast[cast['has_dr_character']]
dr_movie_count = dr_movies[['title', 'year']].drop_duplicates().shape[0]

print("Number of movies with at least one 'Dr.' character:", dr_movie_count)


Number of movies with at least one 'Dr.' character: 18676


### 5. Create a new column indicating whether each movie has a lead role (n=1) played by an actress. 

#### Use apply to achieve this, and then count the number of such movies.


Severely struggled with this equation. All assisstance

In [101]:
# Step 1: Group by movie title and year
grouped = cast.groupby(['title', 'year'])

# Step 2: Define a function to check if any actress has n == 1 in that movie
def has_female_lead(group):
    return ((group['type'] == 'actress') & (group['n'] == 1)).any()

# Step 3: Apply the function to each group
female_lead_flags = grouped.apply(has_female_lead)

# Step 4: Count how many movies have a female lead
num_female_lead_movies = female_lead_flags.sum()

print("Number of movies with a female lead:", num_female_lead_movies)


Number of movies with a female lead: 43691


/var/folders/bs/_y1b6rb96rb21tm4p3r5fpz80000gn/T/ipykernel_83456/2541313339.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  female_lead_flags = grouped.apply(has_female_lead)


In [100]:
cast['has_female_lead'].sum()


np.int64(45614)